In [1]:
#!pip install -q transformers accelerate bitsandbytes
#!pip install -U bitsandbytes
#!pip install -U bitsandbytes==0.43.1
!pip install flash_attn
!pip install flash_attn
!pip install transformers accelerate peft
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 36.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash_attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=256040057 sha256=f25da18657a87fc83dc1bfb8b7751b82246e9db355510226b674fd437c34b5fb
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built flash_attn


In [2]:
torch.cuda.is_available()

False

# 0. Initialization

In [2]:
# train_gsm8k_reinforce.py
import os, math, random, re, json, time
from dataclasses import dataclass
from typing import List, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from datasets import load_dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, get_linear_schedule_with_warmup
)
from peft import LoraConfig, get_peft_model, TaskType

# ----------------------------
# Config
# ----------------------------
class Cfg:
    model_id = "huihui-ai/Huihui-MoE-1B-A0.6B-SFT"  # your pick
    max_prompt_len = 256
    max_new_tokens = 128
    temperature = 0.8
    top_p = 0.95
    do_sample = True
    num_return_sequences = 1             # K completions per prompt
    train_batch_size = 4                 # micro-batch (episodes) per step
    grad_accum_steps = 32                # -> effective batch ~ 128 episodes
    num_updates = 12 #1200                   # ~1–2h depending on GPU
    lr_policy = 1e-5
    lr_value = 5e-5
    weight_decay = 0.01
    warmup_steps = 100
    beta_kl = 0.05                       # starting KL coef
    target_kl_per_token = 0.03           # adaptive target
    beta_kl_up = 1.3
    beta_kl_down = 0.7
    entropy_coef = 1e-3                  # small exploration bonus (optional)
    clip_value = 0.2                     # value clipping (optional)
    log_every = 10
    save_every = 200
    out_dir = "runs_reinforce_kl"
    device = "cuda" if torch.cuda.is_available() else "cpu"
    fp16 = True
    seed = 42
    # LoRA (already applied in your snippet — keep in case you run standalone)
    lora = dict(r=16, alpha=32, target_modules=["up_proj", "down_proj"], dropout=0.05)

random.seed(Cfg.seed)
torch.manual_seed(Cfg.seed)
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
os.makedirs(Cfg.out_dir, exist_ok=True)


In [3]:
# ----------------------------
# Load model/tokenizer (comment out if you already have them)
# ----------------------------
model_id = Cfg.model_id
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

policy = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True
)
policy.config.use_cache = False  # IMPORTANT for training

# Apply LoRA adapters (comment out if you already applied)
lconf = LoraConfig(
    r=Cfg.lora["r"],
    lora_alpha=Cfg.lora["alpha"],
    target_modules=Cfg.lora["target_modules"],
    lora_dropout=Cfg.lora["dropout"],
    bias="none",
    task_type=TaskType.CAUSAL_LM
)
policy = get_peft_model(policy, lconf)

# Reference policy (frozen)
ref_policy = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True
)
ref_policy.eval()
for p in ref_policy.parameters():
    p.requires_grad = False

print("Pretrained model with LoRA layers: \n", policy)
print("-"*100)
# ----------------------------
# Add a small Value Head
# ----------------------------
hidden_size = policy.config.hidden_size
class ValueHead(nn.Module):
    def __init__(self, hs):
        super().__init__()
        self.v = nn.Sequential(
            nn.Linear(hs, hs),
            nn.Tanh(),
            nn.Linear(hs, 1),
        )
    def forward(self, h_last):  # (..., H) -> (..., 1)
        return self.v(h_last).squeeze(-1)

value_head = ValueHead(hidden_size).to(Cfg.device)
print("Value head:\n", value_head)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/127 [00:00<?, ?B/s]

Pretrained model with LoRA layers: 
 PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3MoeForCausalLM(
      (model): Qwen3MoeModel(
        (embed_tokens): Embedding(151936, 1024)
        (layers): ModuleList(
          (0-27): 28 x Qwen3MoeDecoderLayer(
            (self_attn): Qwen3MoeAttention(
              (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
              (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
              (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
              (q_norm): Qwen3MoeRMSNorm((128,), eps=1e-06)
              (k_norm): Qwen3MoeRMSNorm((128,), eps=1e-06)
            )
            (mlp): Qwen3MoeSparseMoeBlock(
              (gate): Linear(in_features=1024, out_features=3, bias=False)
              (experts): ModuleList(
                (0-2): 3 x Qwen3MoeMLP(
                  (gate_proj): Linea

In [4]:
# ----------------------------
# GSM8K dataset and formatting
# ----------------------------
def build_prompt(question: str) -> str:
    # Very simple CoT-style prompt; you can enrich if you want.
    return (
        "You are a careful math tutor. Solve step by step and give the final numeric answer.\n\n"
        f"Question: {question}\n\n"
        "Reasoning:"
    )

gsm = load_dataset("gsm8k", "main")
train_data = gsm["train"]
test_data = gsm["test"]

@dataclass
class Example:
    prompt: str
    gold: str

def gold_answer_from_gsm(gsm_answer: str) -> str:
    # GSM8K answers typically end with a line like "#### 42"
    m = re.search(r"####\s*([^\n]+)", gsm_answer)
    return m.group(1).strip() if m else gsm_answer.strip()

def collate_fn(batch):
    return batch

train_pool: List[Example] = [
    Example(build_prompt(ex["question"]), gold_answer_from_gsm(ex["answer"]))
    for ex in train_data
]
val_pool: List[Example] = [
    Example(build_prompt(ex["question"]), gold_answer_from_gsm(ex["answer"]))
    for ex in test_data.select(range(500))
]

train_loader = DataLoader(train_pool, batch_size=Cfg.train_batch_size, shuffle=True, collate_fn=collate_fn)


README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [5]:
# ----------------------------
# Utilities: numeric parsing & reward
# ----------------------------
num_pat = re.compile(r"-?\d+(?:\.\d+)?(?:/\d+)?")

def parse_numeric(text: str) -> str:
    # 1) \boxed{...}  2) after "Final Answer:"  3) last standalone number  4) GSM8K #### line
    m = re.search(r"\\boxed\{([^}]+)\}", text)
    if m: return m.group(1).strip()
    m = re.search(r"(?:Final Answer|final answer)\s*[:：]\s*([^\n]+)", text, flags=re.I)
    if m: return m.group(1).strip()
    # try GSM8K style in model output
    m = re.search(r"####\s*([^\n]+)", text)
    if m: return m.group(1).strip()
    nums = num_pat.findall("\n".join(text.strip().splitlines()[-3:]))
    return nums[-1] if nums else text.strip()

def equal_numeric(pred: str, gold: str, tol=1e-6) -> bool:
    # try fraction/float normalize
    try:
        from fractions import Fraction
        def to_val(x):
            x = x.replace(",", "").strip()
            if "/" in x and not x.strip().startswith("http"):
                return float(Fraction(x))
            return float(x)
        return abs(to_val(pred) - to_val(gold)) <= tol
    except Exception:
        return pred.strip() == gold.strip()

def reward_fn(gen_text: str, gold_text: str) -> float:
    pred = parse_numeric(gen_text)
    gold = parse_numeric(gold_text)
    r = 1.0 if equal_numeric(pred, gold) else 0.0
    # small shaping: encourage explicit final answer token
    if re.search(r"(Final Answer|####|\\boxed)", gen_text, flags=re.I):
        r += 0.05
    # small length penalty for very long rambling
    if len(gen_text) > 2000:
        r -= 0.05
    return max(min(r, 1.2), -1.0)


In [6]:
# ----------------------------
# Token-level logprobs helper
# ----------------------------
@torch.no_grad()
# do inference on the prompt
def generate_batch(prompts: List[str]):
    inputs = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=Cfg.max_prompt_len
    ).to(Cfg.device)
    gen_out = policy.generate(
        **inputs,
        max_new_tokens=Cfg.max_new_tokens,
        do_sample=Cfg.do_sample,
        temperature=Cfg.temperature,
        top_p=Cfg.top_p,
        num_return_sequences=Cfg.num_return_sequences,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        use_cache=True,          # ← change #1 here (speeds up sampling)
    )

    # Repeat prompts accordingly
    expanded_prompts = []
    for p in prompts:
        expanded_prompts.extend([p] * Cfg.num_return_sequences)
    return expanded_prompts, gen_out

def gather_logprobs(model, input_ids, attention_mask, labels):
    """
    Teacher-forced logprobs for labels tokens.
    Returns: token_logprobs [B, T] where labels!=-100, else 0.
    """
    out = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        labels=None,
        output_hidden_states=True,
        use_cache=False
    )
    logits = out.logits[:, :-1, :]                   # shift for next-token pred
    tgt = input_ids[:, 1:]                           # targets
    mask = attention_mask[:, 1:].bool()
    vocab_logprobs = F.log_softmax(logits, dim=-1)
    tok_lp = vocab_logprobs.gather(-1, tgt.unsqueeze(-1)).squeeze(-1)
    tok_lp = tok_lp * mask  # zero out padding
    return tok_lp, out.hidden_states  # hidden states for value head

def build_lm_inputs(prompts: List[str], completions: List[List[int]]):
    """
    Concatenate prompt + completion token IDs; construct attention masks.
    """
    enc = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=Cfg.max_prompt_len
    )
    input_ids = enc["input_ids"]
    attn = enc["attention_mask"]
    B = input_ids.size(0)
    # append completions
    seqs = []
    masks = []
    comp_ranges = []  # (start_idx, end_idx) for each sequence
    for b in range(B):
        comp = torch.tensor(completions[b], dtype=torch.long)
        seq = torch.cat([input_ids[b], comp], dim=0)
        mask = torch.cat([attn[b], torch.ones_like(comp)], dim=0)
        seqs.append(seq)
        masks.append(mask)
        comp_ranges.append((input_ids[b].size(0), seq.size(0)))  # generated span
    max_len = max(s.size(0) for s in seqs)
    padded_ids = torch.full((B, max_len), tokenizer.pad_token_id, dtype=torch.long)
    padded_mask = torch.zeros((B, max_len), dtype=torch.long)
    for b, (s, m) in enumerate(zip(seqs, masks)):
        padded_ids[b, :s.size(0)] = s
        padded_mask[b, :m.size(0)] = m
    return padded_ids.to(Cfg.device), padded_mask.to(Cfg.device), comp_ranges


In [7]:
from collections import deque
from tqdm.auto import tqdm

# ----------------------------
# Optimizers & AMP
# ----------------------------
trainable_policy = [p for n,p in policy.named_parameters() if p.requires_grad]
opt = torch.optim.AdamW(
    [{"params": trainable_policy, "lr": Cfg.lr_policy},
     {"params": value_head.parameters(), "lr": Cfg.lr_value}],
    weight_decay=Cfg.weight_decay, betas=(0.9, 0.95)
)
scaler = torch.cuda.amp.GradScaler(enabled=Cfg.fp16)

# Dummy scheduler over updates (optional)
scheduler = get_linear_schedule_with_warmup(opt, Cfg.warmup_steps, Cfg.num_updates)

# ----------------------------
# Training loop (REINFORCE + Baseline + KL)
# ----------------------------
def adaptive_kl_beta(beta, observed_kl_tok):
    if observed_kl_tok > Cfg.target_kl_per_token * 1.5:
        beta *= Cfg.beta_kl_up
    elif observed_kl_tok < Cfg.target_kl_per_token * 0.5:
        beta *= Cfg.beta_kl_down
    return float(beta)

policy.train()
value_head.train()

global_step = 0
update = 0
beta_kl = Cfg.beta_kl
best_em = 0.0

t0 = time.time()
#MR: go for one iteration only
while update < Cfg.num_updates:
    cntr = 0
    for batch in train_loader:
        cntr += 1
        print("Analysing the batch #{}".format(cntr))
        # 1) Rollouts: generate K completions per prompt
        prompts = [ex.prompt for ex in batch]  # len = micro-batch
        golds   = [ex.gold for ex in batch]

        with torch.no_grad():
            expanded_prompts, gen_ids = generate_batch(prompts)  # size: micro * K
            # split into per-sequence lists of completion ids (excluding prompt)
            # We need the raw completion part: easiest is to recompute tokenization boundaries.
            # Instead, take full sequence and later isolate generated span via comp_ranges.

        # 2) For each generated sequence, compute reward
        texts = tokenizer.batch_decode(gen_ids, skip_special_tokens=True)
        # Extract only the completion text (after the prompt) for reward readability
        expanded_golds = []
        for g in golds:
            expanded_golds.extend([g] * Cfg.num_return_sequences)

        rewards = torch.tensor([reward_fn(txt, gold) for txt, gold in zip(texts, expanded_golds)],
                               dtype=torch.float, device=Cfg.device)
        # Group baseline by prompt (optional, helps variance)
        grouped_means = []
        for i in range(0, len(rewards), Cfg.num_return_sequences):
            grouped_means.extend([rewards[i:i+Cfg.num_return_sequences].mean().item()] * Cfg.num_return_sequences)
        grouped_means = torch.tensor(grouped_means, device=Cfg.device)
        rewards_centered = rewards - grouped_means  # group baseline

        # 3) Build LM inputs (prompt + completion) to compute logprobs/values/KL
        # We need token ids for the generated part; here we re-tokenize the FINAL texts.
        enc_full = tokenizer(
            expanded_prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=Cfg.max_prompt_len
        )
        # Concatenate the generated ids per sequence to the corresponding prompt
        # We must split gen_ids back into sequences:
        seq_len = gen_ids.size(1)
        completions = [gen_ids[i].tolist()[enc_full["input_ids"][i % len(batch)].ne(tokenizer.pad_token_id).sum().item():]
                       for i in range(gen_ids.size(0))]  # heuristic; robust enough
        input_ids, attn_mask, comp_ranges = build_lm_inputs(expanded_prompts, completions)

        # 4) Compute token logprobs for current and reference, and hidden states for value
        with torch.cuda.amp.autocast(enabled=Cfg.fp16):
            tok_lp_pi, hs_pi = gather_logprobs(policy, input_ids, attn_mask, labels=None)
            tok_lp_ref, _    = gather_logprobs(ref_policy, input_ids, attn_mask, labels=None)

        # mask to only generated tokens
        gen_mask = torch.zeros_like(attn_mask)
        for b, (s, e) in enumerate(comp_ranges):
            gen_mask[b, s:e] = 1
        # align to shifted positions (since tok_lp_* are for positions 1..T)
        gen_mask_shifted = gen_mask[:, 1:].bool()

        # sequence logprob over generated tokens
        seq_logprob = (tok_lp_pi * gen_mask_shifted).sum(dim=1)
        seq_logprob_ref = (tok_lp_ref * gen_mask_shifted).sum(dim=1)

        # tokenwise kl approximation on sampled tokens: kl_tok ≈ logπ - logπ_ref
        kl_tok = ((tok_lp_pi - tok_lp_ref) * gen_mask_shifted).sum(dim=1)
        # average KL per generated token (for monitoring/adaptation)
        gen_tok_counts = gen_mask_shifted.sum(dim=1).clamp_min(1)
        kl_per_tok = (kl_tok / gen_tok_counts).mean().item()

        # Value prediction at EOS (take last hidden state for the last token)
        # hs_pi is a tuple of hidden states per layer; take final layer
        last_h = hs_pi[-1]  # [B, T, H]
        # index last non-pad position
        last_idx = attn_mask.sum(dim=1) - 1
        h_last = last_h[torch.arange(last_h.size(0), device=Cfg.device), last_idx]
        values = value_head(h_last)  # [B]

        # 5) Compute advantages and normalize
        with torch.no_grad():
            advantages = rewards_centered - values
            # batch norm for stability
            adv_mean = advantages.mean()
            adv_std = advantages.std().clamp_min(1e-6)
            advantages = (advantages - adv_mean) / adv_std
            advantages = advantages.clamp(-3.0, 3.0)

        # 6) Losses
        with torch.cuda.amp.autocast(enabled=Cfg.fp16):
            # Policy loss (REINFORCE): -A * logπ(y|x)
            L_pg = -(advantages.detach() * seq_logprob).mean()
            # KL penalty
            L_kl = beta_kl * (kl_tok.mean())
            # Entropy bonus (encourage exploration): need token entropy on generated span
            # Quick proxy: negative mean logprob of sampled tokens (higher => more entropy)
            entropy_bonus = -Cfg.entropy_coef * (tok_lp_pi[gen_mask_shifted].mean())
            # Value loss with clipping (optional)
            value_target = rewards_centered
            v_clipped = values + (values - values.detach()).clamp(-Cfg.clip_value, Cfg.clip_value)
            L_v1 = F.mse_loss(values, value_target)
            L_v2 = F.mse_loss(v_clipped, value_target)
            L_v = torch.max(L_v1, L_v2)

            loss = L_pg + L_kl + 0.5 * L_v + entropy_bonus

        scaler.scale(loss / Cfg.grad_accum_steps).backward()
        global_step += 1

        if global_step % Cfg.grad_accum_steps == 0:
            scaler.unscale_(opt)
            torch.nn.utils.clip_grad_norm_(policy.parameters(), 1.0)
            torch.nn.utils.clip_grad_norm_(value_head.parameters(), 1.0)
            scaler.step(opt)
            scaler.update()
            scheduler.step()
            opt.zero_grad(set_to_none=True)

            update += 1

            # adapt KL beta toward target KL/token
            beta_kl = adaptive_kl_beta(beta_kl, kl_per_tok)

            if update % Cfg.log_every == 0:
                # quick EM on this micro-batch
                em = (rewards >= 1.0).float().mean().item()
                print(f"[upd {update:04d}] loss {loss.item():.3f} | L_pg {L_pg.item():.3f} "
                      f"| L_kl {L_kl.item():.3f} | L_v {L_v.item():.3f} | ent {(-entropy_bonus).item():.3f} "
                      f"| KL/tok {kl_per_tok:.4f} | β_KL {beta_kl:.4f} | EM {em:.3f}")

            if update % Cfg.save_every == 0:
                save_dir = os.path.join(Cfg.out_dir, f"step_{update}")
                os.makedirs(save_dir, exist_ok=True)
                # Save LoRA adapters and value head
                policy.save_pretrained(save_dir)
                torch.save(value_head.state_dict(), os.path.join(save_dir, "value_head.pt"))
                tokenizer.save_pretrained(save_dir)
                print(f"Saved checkpoint to {save_dir}")
            #MR: stop after one iteration
            break
            if update >= Cfg.num_updates:
                break

t1 = time.time()
print(f"Training finished in {(t1 - t0)/60:.1f} min")


/tmp/ipython-input-1205827342.py:13: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=Cfg.fp16)
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Analysing the batch #1


/tmp/ipython-input-1205827342.py:86: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=Cfg.fp16):
/tmp/ipython-input-1205827342.py:125: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=Cfg.fp16):
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Analysing the batch #2


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

Analysing the batch #30


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Analysing the batch #31


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Analysing the batch #32
Training finished in 8.7 min


In [11]:
print(policy)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3MoeForCausalLM(
      (model): Qwen3MoeModel(
        (embed_tokens): Embedding(151936, 1024)
        (layers): ModuleList(
          (0-27): 28 x Qwen3MoeDecoderLayer(
            (self_attn): Qwen3MoeAttention(
              (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
              (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
              (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
              (q_norm): Qwen3MoeRMSNorm((128,), eps=1e-06)
              (k_norm): Qwen3MoeRMSNorm((128,), eps=1e-06)
            )
            (mlp): Qwen3MoeSparseMoeBlock(
              (gate): Linear(in_features=1024, out_features=3, bias=False)
              (experts): ModuleList(
                (0-2): 3 x Qwen3MoeMLP(
                  (gate_proj): Linear(in_features=1024, out_features=3072

In [10]:
# ----------------------------
# Quick evaluation on a small dev slice
# ----------------------------
policy.eval()
value_head.eval()

def eval_em(n_eval=40):
    idxs = random.sample(range(len(val_pool)), n_eval)
    prompts = [val_pool[i].prompt for i in idxs]
    golds = [val_pool[i].gold for i in idxs]
    all_preds = []
    with torch.no_grad():
        for i in tqdm(range(0, n_eval, 8)):
            chunk = prompts[i:i+8]
            enc = tokenizer(chunk, return_tensors="pt", padding=True, truncation=True, max_length=Cfg.max_prompt_len).to(Cfg.device)
            outs = policy.generate(**enc, max_new_tokens=Cfg.max_new_tokens, do_sample=False, temperature=0.0, use_cache=True)
            texts = tokenizer.batch_decode(outs, skip_special_tokens=True)
            all_preds.extend(texts)
    hits = 0
    for pred, gold in zip(all_preds, golds):
        hits += int(equal_numeric(parse_numeric(pred), parse_numeric(gold)))
    em = hits / n_eval
    print(f"Dev EM@{n_eval}: {em:.3f}")
    return em

eval_em()


  0%|          | 0/5 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Dev EM@40: 0.050


0.05